# ump

> Untargeted Metabolomics Preprocessing

In [ ]:
#| default_exp ump

In [ ]:
#| exports
import os
import urllib.request
import pyopenms as oms


The universal workflow for untargeted `metabolomics` always consists of *feature detection* in the individual MS sample files and their linkage to *consensus features* with common m/z and retention time values. 

In addition, there are optional steps such as adduct detection and annotation of `features` with associated `MS2` spectra.



download two example `mzML` files.


In [ ]:
gh = "https://raw.githubusercontent.com/OpenMS/pyopenms-docs/master"
URL1 = gh + "/src/data/Metabolomics_1.mzML"
URL2 = gh + "/src/data/Metabolomics_2.mzML"
PROJ_DIR = 'ump/'

# create project directory if it doesn't exist
if not os.path.exists(PROJ_DIR):
    os.makedirs(PROJ_DIR)

# download data to project directory if it doesn't exist
if not os.path.exists(PROJ_DIR + 'Metabolomics_1.mzML'):
    urllib.request.urlretrieve(URL1, PROJ_DIR + 'Metabolomics_1.mzML')
if not os.path.exists(PROJ_DIR + 'Metabolomics_2.mzML'):
    urllib.request.urlretrieve(URL2, PROJ_DIR + 'Metabolomics_2.mzML')

In [ ]:
# change working directory to project directory
os.chdir(PROJ_DIR)

In [ ]:
!pwd

/home/ma/git/comics/nbs/ump


In [ ]:
mzML_files = ["Metabolomics_1.mzML", "Metabolomics_2.mzML"]


For each `mzML` file do mass trace, elution peak and features detection.


In [ ]:
#| export

def process_files(mzML_files):
    """
    Process a list of mzML files and return a list of feature maps.
    
    Parameters:
    mzML_files (list): A list of mzML file paths to be processed.
    
    Returns:
    list: A list of feature maps.
    """
    feature_maps = []
    for file in mzML_files:
        print("Processing file:", file)
        # load mzML file into MSExperiment
        exp = oms.MSExperiment()
        oms.MzMLFile().load(file, exp)  # load each mzML file to an OpenMS file format (MSExperiment)

        # mass trace detection
        mass_traces = ([])  # introduce an empty list where the mass traces will be loaded
        mtd = oms.MassTraceDetection()
        mtd_par = (mtd.getDefaults())  # get the default parameters in order to edit them
        mtd_par.setValue("mass_error_ppm", 10.0)  # high-res instrument, orbitraps
        mtd_par.setValue("noise_threshold_int", 1.0e04)  # data-dependent (usually works for orbitraps)
        mtd.setParameters(mtd_par)  # set the new parameters
        mtd.run(exp, mass_traces, 0)  # run mass trace detection

        # elution peak detection
        mass_traces_deconvol = []
        epd = oms.ElutionPeakDetection()
        epd_par = epd.getDefaults()
        epd_par.setValue(
            "width_filtering", "fixed"
        )  # The fixed setting filters out mass traces outside the [min_fwhm: 1.0, max_fwhm: 60.0] interval
        epd.setParameters(epd_par)
        epd.detectPeaks(mass_traces, mass_traces_deconvol)

        # feature detection
        feature_map = oms.FeatureMap()  # output features
        chrom_out = []  # output chromatograms
        ffm = oms.FeatureFindingMetabo()
        ffm_par = ffm.getDefaults()
        ffm_par.setValue(
            "remove_single_traces", "true"
        )  # remove mass traces without satellite isotopic traces
        ffm.setParameters(ffm_par)
        ffm.run(mass_traces_deconvol, feature_map, chrom_out)
        feature_map.setUniqueIds()  # Assigns a new, valid unique id per feature
        feature_map.setPrimaryMSRunPath(
            [file.encode()]
        )  # Sets the file path to the primary MS run (usually the mzML file)
        feature_maps.append(feature_map)
    return feature_maps

In [ ]:
# process files
feature_maps = process_files(mzML_files)

Processing file: Metabolomics_1.mzML
Progress of 'mass trace detection':
-- done [took 0.01 s (CPU), 0.01 s (Wall)] -- 
Progress of 'elution peak detection':
-- done [took 0.07 s (CPU), 0.00 s (Wall)] -- 
Progress of 'assembling mass traces to features':
Loading metabolite isotope model with 5% RMS error
-- done [took 0.29 s (CPU), 0.00 s (Wall)] -- 
Processing file: Metabolomics_2.mzML
Progress of 'mass trace detection':
-- done [took 0.01 s (CPU), 0.01 s (Wall)] -- 
Progress of 'elution peak detection':
-- done [took 0.01 s (CPU), 0.00 s (Wall)] -- 
Progress of 'assembling mass traces to features':
-- done [took 0.29 s (CPU), 0.00 s (Wall)] -- 


retention times alignment

based on the `feature map` with the highest number of features (reference map).


In [ ]:
# use as reference for alignment, the file with the largest number of features
# (works well if you have a pooled QC for example)
ref_index = feature_maps.index(sorted(feature_maps, key=lambda x: x.size())[-1])

aligner = oms.MapAlignmentAlgorithmPoseClustering()

trafos = {}

# parameter optimization
aligner_par = aligner.getDefaults()
aligner_par.setValue("max_num_peaks_considered", -1)  # infinite
aligner_par.setValue(
    "pairfinder:distance_MZ:max_difference", 10.0
)  # Never pair features with larger m/z distance
aligner_par.setValue("pairfinder:distance_MZ:unit", "ppm")
aligner.setParameters(aligner_par)
aligner.setReference(feature_maps[ref_index])

for feature_map in feature_maps[:ref_index] + feature_maps[ref_index + 1 :]:
    trafo = oms.TransformationDescription()  # save the transformed data points
    aligner.align(feature_map, trafo)
    trafos[feature_map.getMetaValue("spectra_data")[0].decode()] = trafo
    transformer = oms.MapAlignmentTransformer()
    transformer.transformRetentionTimes(feature_map, trafo, True)


Align `mzML` files aligment based on FeatureMap alignment (optional, only for GNPS).

GNPS is a web-based mass spectrometry ecosystem that aims to be an open-access knowledge base for community-wide organization and sharing of raw, processed or identified tandem mass (MS/MS) spectrometry data.


In [ ]:

for file in mzML_files:
    exp = oms.MSExperiment()
    oms.MzMLFile().load(file, exp)
    exp.sortSpectra(True)
    exp.setMetaValue("mzML_path", file)
    if file not in trafos.keys():
        oms.MzMLFile().store(file[:-5] + "_aligned.mzML", exp)
        continue
    transformer = oms.MapAlignmentTransformer()
    trafo_description = trafos[file]
    transformer.transformRetentionTimes(exp, trafo_description, True)
    oms.MzMLFile().store(file[:-5] + "_aligned.mzML", exp)
mzML_files = [file[:-5] + "_aligned.mzML" for file in mzML_files]


Map `MS2` spectra to features as PeptideIdentification objects (optional, only for GNPS).


In [ ]:
feature_maps_mapped = []
use_centroid_rt = False
use_centroid_mz = True
mapper = oms.IDMapper()
for file in mzML_files:
    exp = oms.MSExperiment()
    oms.MzMLFile().load(file, exp)
    for i, feature_map in enumerate(feature_maps):
        if feature_map.getMetaValue("spectra_data")[
            0
        ].decode() == exp.getMetaValue("mzML_path"):
            peptide_ids = []
            protein_ids = []
            mapper.annotate(
                feature_map,
                peptide_ids,
                protein_ids,
                use_centroid_rt,
                use_centroid_mz,
                exp,
            )
            fm_new = oms.FeatureMap(feature_map)
            fm_new.clear(False)
            # set unique identifiers to protein and peptide identifications
            prot_ids = []
            if len(feature_map.getProteinIdentifications()) > 0:
                prot_id = feature_map.getProteinIdentifications()[0]
                prot_id.setIdentifier(f"Identifier_{i}")
                prot_ids.append(prot_id)
            fm_new.setProteinIdentifications(prot_ids)
            for feature in feature_map:
                pep_ids = []
                for pep_id in feature.getPeptideIdentifications():
                    pep_id.setIdentifier(f"Identifier_{i}")
                    pep_ids.append(pep_id)
                feature.setPeptideIdentifications(pep_ids)
                fm_new.push_back(feature)
            feature_maps_mapped.append(fm_new)
feature_maps = feature_maps_mapped


IDMapper warning: at least one feature has no convex hull - using centroid coordinates for matching
1052 spectra and 1 chromatograms stored.
<Loading metabolite isotope model with 5% RMS error> occurred 2 times
963 spectra and 1 chromatograms stored.
Unassigned peptides: 0
Peptides assigned to exactly one feature: 0
Peptides assigned to multiple features: 0
Unassigned and unidentified precursors: 718
Unidentified precursor assigned to exactly one feature: 0
Unidentified precursor assigned to multiple features: 0
Feature annotation with identifications:
    no ID: 74
    single ID: 0
    multiple IDs (identical): 0
    multiple IDs (divergent): 0


Unassigned peptides: 0
Peptides assigned to exactly one feature: 0
Peptides assigned to multiple features: 0
Unassigned and unidentified precursors: 634
Unidentified precursor assigned to exactly one feature: 0
Unidentified precursor assigned to multiple features: 0
Feature annotation with identifications:
    no ID: 70
    single ID: 0
    m


Detect adducts (optional, only for SIRIUS and GNPS Ion Identity
Molecular Networking).


In [ ]:
feature_maps_adducts = []
for feature_map in feature_maps:
    mfd = oms.MetaboliteFeatureDeconvolution()
    mdf_par = mfd.getDefaults()
    mdf_par.setValue(
        "potential_adducts",
        [
            b"H:+:0.4",
            b"Na:+:0.2",
            b"NH4:+:0.2",
            b"H-1O-1:+:0.1",
            b"H-3O-2:+:0.1",
        ],
    )
    mfd.setParameters(mdf_par)
    feature_map_adduct = oms.FeatureMap()
    mfd.compute(feature_map, feature_map_adduct, oms.ConsensusMap(), oms.ConsensusMap())
    feature_maps_adducts.append(feature_map_adduct)
feature_maps = feature_maps_adducts

# for SIRIUS store the feature maps as featureXML files!
for feature_map in feature_maps:
    oms.FeatureXMLFile().store(
        feature_map.getMetaValue("spectra_data")[0].decode()[:-4]
        + "featureXML",
        feature_map,
    )


MassExplainer table size: 312
Generating Masses with threshold: -6.90776 ...
done
4 of 17 valid net charge compomer results did not pass the feature charge constraints
Inferring edges raised edge count from 19 to 19
Found 19 putative edges (of 53) and avg hit-size of 1.21053
Using solver 'coinor' ...
Optimal solution found!
 Branch and cut took 0.001334 seconds,  with objective value: 0.6032.
ILP score is: 0.6032
Agreeing charges: 26/26
MassExplainer table size: 312
Generating Masses with threshold: -6.90776 ...
done
28 of 85 valid net charge compomer results did not pass the feature charge constraints
Inferring edges raised edge count from 80 to 80
Found 80 putative edges (of 165) and avg hit-size of 1.5125
Using solver 'coinor' ...
Optimal solution found!
 Branch and cut took 0.005414 seconds,  with objective value: 1.59.
ILP score is: 1.59
Agreeing charges: 86/86


Link features in a ConsensusMap


In [ ]:

feature_grouper = oms.FeatureGroupingAlgorithmKD()

consensus_map = oms.ConsensusMap()
file_descriptions = consensus_map.getColumnHeaders()

for i, feature_map in enumerate(feature_maps):
    file_description = file_descriptions.get(i, oms.ColumnHeader())
    file_description.filename = os.path.basename(
        feature_map.getMetaValue("spectra_data")[0].decode()
    )
    file_description.size = feature_map.size()
    file_descriptions[i] = file_description

feature_grouper.group(feature_maps, consensus_map)
consensus_map.setColumnHeaders(file_descriptions)
consensus_map.setUniqueIds()
oms.ConsensusXMLFile().store("FeatureMatrix.consensusXML", consensus_map)


Progress of 'computing RT transformations':
.
<IDMapper warning: at least one feature has no convex hull - using centroid coordinates for matching> occurred 2 times
..
.
<..> occurred 2 times
-- done [took 0.00 s (CPU), 0.00 s (Wall)] -- 
Progress of 'linking features':
-- done [took 0.01 s (CPU), 0.00 s (Wall)] -- 



To get a final feature matrix in a table format, export the
`:consensus features<consensus feature>` in a `pandas DataFrame`.


In [ ]:
df = consensus_map.get_df()
df

,sequence,charge,RT,mz,quality,Metabolomics_1.mzML,Metabolomics_2.mzML
id,,,,,,,
18357009966950325368,None,1,498.987864,301.075331,0.002225,2.007466e+06,1.974032e+06
16904852559167011933,None,1,498.987864,279.093382,0.016148,1.321831e+07,1.407178e+07
16309913421659575652,None,1,544.292638,209.153597,0.001844,1.468729e+06,1.684743e+06
1658001803149532672,None,1,319.982463,449.285023,0.000085,2.866735e+06,0.000000e+00
5797594689815113151,None,1,325.955992,473.344926,0.000082,2.727014e+06,0.000000e+00
...,...,...,...,...,...,...,...
17509933937483595593,None,1,396.384892,651.374876,0.023374,0.000000e+00,9.109984e+06
17758318573007799102,None,1,352.738971,687.395798,0.001786,0.000000e+00,7.517256e+05
3656125905357566612,None,1,439.999344,672.404288,0.004309,0.000000e+00,1.830204e+06


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()